In [0]:
#!import "./Scopes"

# Other Methods

In [0]:
//output ComputeSolvency(inputs){
//     var SolvencyStorage =
//     SolvencyStorage.InitializeAsync();
//     var solvencyUniverse = 
//     var identites = 
//     solvencyUniverse.GetsScope<ComputeSolvencyVariables>(identities);
// }

In [0]:
public async Task<ActivityLog> ComputePillarAsync(ImportArgs args, IWorkspace workspace, IWorkspace workspaceToCompute, bool saveRawVariables, string pillar)
{
    // //switch pillar {
    //     "Solvency" => ComputeSolvency(),
    // }
    return null;
}

# Generic Cashflow Defition

In [0]:
public async Task<ActivityLog> ParseCashflowsDefinitionToWorkspaceAsync(IDataSet dataSet, ImportArgs args, IWorkspace workspace, IDataSource targetDataSource, string pillar = null)
{
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(targetDataSource).DisableInitialization<Cashflow>().DisableInitialization<IfrsVariable>());
    
    Activity.Start();
    // var parsingStorage = new ParsingStorage(args, targetDataSource, workspace); //
    // await parsingStorage.InitializeAsync();
    
    var TargetPartitionByReportingNodeAndPeriod = (await workspace.Query<PartitionByReportingNodeAndPeriod>()
        .Where(p => p.ReportingNode == args.ReportingNode && p.Year == args.Year && p.Month == args.Month && p.Scenario == args.Scenario).ToArrayAsync())
        .SingleOrDefault();
            
    if(Activity.HasErrors()) return Activity.Finish();
    
    var hasAccidentYearColumn = new Dictionary<string,bool>(){};
    if(dataSet.Tables.Contains(nameof(BasicCashflow))) hasAccidentYearColumn[nameof(BasicCashflow)] = dataSet.Tables[nameof(BasicCashflow)].Columns.Any(x => x.ColumnName == nameof(Cashflow.AccidentYear));
    if(dataSet.Tables.Contains(nameof(BoundedCashflow))) hasAccidentYearColumn[nameof(BoundedCashflow)] = dataSet.Tables[nameof(BoundedCashflow)].Columns.Any(x => x.ColumnName == nameof(Cashflow.AccidentYear));
    if(dataSet.Tables.Contains(nameof(ReferencedCashflow))) hasAccidentYearColumn[nameof(ReferencedCashflow)] = dataSet.Tables[nameof(ReferencedCashflow)].Columns.Any(x => x.ColumnName == nameof(Cashflow.AccidentYear));

    var importLog = await Import.FromDataSet(dataSet)
        .WithType<BasicCashflow> ( (dataset, datarow) => {
            //TODO extract these codes to small modular functions
            int? accidentYear = default;
            if(hasAccidentYearColumn[nameof(BasicCashflow)] && datarow.Field<string>(nameof(Cashflow.AccidentYear)) != null)
                if(Int32.TryParse(datarow.Field<string>(nameof(Cashflow.AccidentYear)), out var parsedAccidentYear))
                    accidentYear = (int?)parsedAccidentYear;
            
            var values = datarow.Table.Columns.Where(c => c.ColumnName.StartsWith(nameof(Cashflow.Values))).OrderBy(c => c.ColumnName.Length).ThenBy(c => c.ColumnName)
                                .Select(x => datarow.Field<string>(x.ColumnName).CheckStringForExponentialAndConvertToDouble()).ToArray();
            
            if (!Enum.TryParse(datarow.Field<string>(nameof(BasicCashflow.Shape)), out Shape shape)) { ApplicationMessage.Log(Error.Generic, "Wrong Shape - Add error message"); return null; };
            
            var item = new BasicCashflow {
                DataNode = datarow.Field<string>(nameof(Cashflow.DataNode)),
                AmountType = datarow.Field<string>(nameof(Cashflow.AmountType)),
                AccidentYear = accidentYear,
                Shape = shape,
                Pillar = pillar,
                Partition = TargetPartitionByReportingNodeAndPeriod.Id,
                Values = values,
                InitialValue = datarow.Field<string>(nameof(BasicCashflow.InitialValue)).CheckStringForExponentialAndConvertToDouble(),
                Length = Int32.TryParse(datarow.Field<string>(nameof(BasicCashflow.Length)), out var length) ? length : default, //TODO Add ParseAndValidateIntProperty
                Shift = Int32.TryParse(datarow.Field<string>(nameof(BasicCashflow.Shift)), out var shift) ? shift : default,
            };
            return item;
        }, nameof(BasicCashflow))
    
    .WithType<BoundedCashflow> ( (dataset, datarow) => {
            int? accidentYear = default;
            if(hasAccidentYearColumn[nameof(BoundedCashflow)] && datarow.Field<string>(nameof(Cashflow.AccidentYear)) != null)
                if(Int32.TryParse(datarow.Field<string>(nameof(Cashflow.AccidentYear)), out var parsedAccidentYear))
                    accidentYear = (int?)parsedAccidentYear;
            
            var values = datarow.Table.Columns.Where(c => c.ColumnName.StartsWith(nameof(Cashflow.Values))).OrderBy(c => c.ColumnName.Length).ThenBy(c => c.ColumnName)
                                .Select(x => datarow.Field<string>(x.ColumnName).CheckStringForExponentialAndConvertToDouble()).ToArray();
            
            if (!Enum.TryParse(datarow.Field<string>(nameof(BoundedCashflow.Shape)), out Shape shape)) { ApplicationMessage.Log(Error.Generic, "Wrong Shape - Add error message"); return null; };
            var item = new BoundedCashflow {
                DataNode = datarow.Field<string>(nameof(Cashflow.DataNode)),
                AmountType = datarow.Field<string>(nameof(Cashflow.AmountType)),
                AccidentYear = accidentYear,
                Shape = shape,
                Pillar = pillar,
                Partition = TargetPartitionByReportingNodeAndPeriod.Id,
                Values = values,
                InitialValue = datarow.Field<string>(nameof(BasicCashflow.InitialValue)).CheckStringForExponentialAndConvertToDouble(),
                Length = Int32.TryParse(datarow.Field<string>(nameof(BasicCashflow.Length)), out var length) ? length : default, //TODO Add ParseAndValidateIntProperty
                Shift = Int32.TryParse(datarow.Field<string>(nameof(BasicCashflow.Shift)), out var shift) ? shift : default,
                Minimum = datarow.Field<string>(nameof(BoundedCashflow.Minimum)).CheckStringForExponentialAndConvertToDouble(),
                Maximum = datarow.Field<string>(nameof(BoundedCashflow.Maximum)).CheckStringForExponentialAndConvertToDouble(),
            };
            return item;
        }, nameof(BoundedCashflow))
    
    .WithType<ReferencedCashflow> ( (dataset, datarow) => {
            int? accidentYear = default;
            if(hasAccidentYearColumn[nameof(ReferencedCashflow)] && datarow.Field<string>(nameof(Cashflow.AccidentYear)) != null)
                if(Int32.TryParse(datarow.Field<string>(nameof(Cashflow.AccidentYear)), out var parsedAccidentYear))
                    accidentYear = (int?)parsedAccidentYear;
            
            var values = datarow.Table.Columns.Where(c => c.ColumnName.StartsWith(nameof(Cashflow.Values))).OrderBy(c => c.ColumnName.Length).ThenBy(c => c.ColumnName)
                                .Select(x => datarow.Field<string>(x.ColumnName).CheckStringForExponentialAndConvertToDouble()).ToArray();
            
            if (!Enum.TryParse(datarow.Field<string>(nameof(ReferencedCashflow.Shape)), out Shape shape)) { ApplicationMessage.Log(Error.Generic, "Wrong Shape - Add error message"); return null; };
            var item = new ReferencedCashflow {
                DataNode = datarow.Field<string>(nameof(Cashflow.DataNode)),
                AmountType = datarow.Field<string>(nameof(Cashflow.AmountType)),
                AccidentYear = accidentYear,
                Shape = shape,
                Pillar = pillar,
                Partition = TargetPartitionByReportingNodeAndPeriod.Id,
                Values = values,
                InitialValue = datarow.Field<string>(nameof(BasicCashflow.InitialValue)).CheckStringForExponentialAndConvertToDouble(),
                Length = Int32.TryParse(datarow.Field<string>(nameof(BasicCashflow.Length)), out var length) ? length : default,//TODO Add ParseAndValidateIntProperty
                Shift = Int32.TryParse(datarow.Field<string>(nameof(BasicCashflow.Shift)), out var shift) ? shift : default,
                ReferenceAmountType = datarow.Field<string>(nameof(ReferencedCashflow.ReferenceAmountType)),
            };
            return item;
        }, nameof(ReferencedCashflow))
        
    .WithTarget(workspace).ExecuteAsync();
    //await workspace.ExtendParsedVariables(parsingStorage.HierarchyCache);
    //await workspace.ValidateForMandatoryAocSteps(dataSet, parsingStorage.MandatoryAocSteps);
    //await workspace.ValidateForDataNodeStateActiveAsync<RawVariable>(parsingStorage.DataNodeDataBySystemName);
    return Activity.Finish().Merge(importLog);
}

In [0]:
public async Task<ActivityLog> CashflowsDefinitionImporterAsync(IDataSet dataSet, IDataSource targetDataSource, string importFormat, string pillar = null)
{
    Activity.Start();
    var primaryArgs = await GetArgsAndCommitPartitionAsync<PartitionByReportingNodeAndPeriod>(dataSet, targetDataSource) with {ImportFormat = importFormat};
    if(Activity.HasErrors()) return Activity.Finish();

    var allArgs = await GetAllArgsAsync(primaryArgs, targetDataSource, importFormat);

    if(dataSet.Tables.Contains(nameof(BasicCashflow))) await DataNodeFactoryAsync(dataSet, nameof(BasicCashflow), primaryArgs, targetDataSource);
    if(dataSet.Tables.Contains(nameof(BoundedCashflow))) await DataNodeFactoryAsync(dataSet, nameof(BoundedCashflow), primaryArgs, targetDataSource);
    if(dataSet.Tables.Contains(nameof(ReferencedCashflow))) await DataNodeFactoryAsync(dataSet, nameof(ReferencedCashflow), primaryArgs, targetDataSource);
    if(Activity.HasErrors()) return Activity.Finish();

    var workspace = Workspace.CreateNew();
    var log = await ParseCashflowsDefinitionToWorkspaceAsync(dataSet, primaryArgs, workspace, targetDataSource, pillar);
    if(log.Errors.Any()) return Activity.Finish().Merge(log);

    var workspaceToCompute = Workspace.CreateNew();
    workspaceToCompute.Initialize(x => x.FromSource(targetDataSource));
    // if (Debug.Enable) 
    // {
    //     if(primaryArgs.Scenario == null) //What is the type here?
    //         await workspace.DeleteAsync(await workspace.Query<Cashflow>().Where(rv => rv.Values.Sum(x => Math.Abs(x)) < Precision).ToArrayAsync());
    //     var partition = (Guid)(await workspace.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(primaryArgs));
    //     await workspace.CommitToAsync<Cashflow, PartitionByReportingNodeAndPeriod>(workspaceToCompute, partition, snapshot : true);
    // }  
    // else 
    //     foreach (var args in allArgs) {
    //         log = log.Merge(await ComputeAsync(args, workspace, workspaceToCompute, args == primaryArgs, pillar));
    //         if(log.Errors.Any()) return Activity.Finish().Merge(log);
    //     }

    //await workspaceToCompute.CommitToTargetAsync(targetDataSource); //original code
    await workspace.CommitToTargetAsync(targetDataSource);
    workspace.Dispose();
    workspaceToCompute.Dispose();
    return Activity.Finish().Merge(log); 
}

In [0]:
Import.DefineFormat("CashflowDefinition", async (options, dataSet) => {
    Activity.Start();
    var log = await CashflowsDefinitionImporterAsync(dataSet, options.TargetDataSource, "CashflowDefinition");
    return Activity.Finish().Merge(log); 
});

# Solvency Cashflow Defition

In [0]:
Import.DefineFormat(SolvencyImportFormats.SolvencyCashflowDefinition, async (options, dataSet) => {
    Activity.Start();
    var log = await CashflowsDefinitionImporterAsync(dataSet, options.TargetDataSource, SolvencyImportFormats.SolvencyCashflowDefinition, "Solvency");
    return Activity.Finish().Merge(log); 
});